<h1 align="center"><b>Preprocesamiento</b>
<hr>

## Importación de paquetes de NLTK y TextBlob

1. Importamos word_tokenize, stopwords, PorterStemmer y TextBlob
> NLTK es una librería especializada en Lenguaje Natural, por lo que nos permite hacer procesamiento de texto para clasificación, tokenización, derivación, etiquetado, análisis de sentimientos etc.
> <br> **word_tokenize:** Nos permite dividir una cadena (string) en substrings: por ejemplo "Este es un texto" a ["esto", "es", "un", "texto"]
> <br> **stopwords:** Nos permite eliminar palabras innecesarias de un texto
> <br> **PorterStemmer:** Es un algoritmo que quita sufijos de las palabras, de tal manera que la palabra quede en su estado primitivo, o raiz, por lo que fasilitaría la clasificación ya que en un tweet podriamos repetir una sola raiz, mas no la misma palabra.
> <br> **TextBlob:** Es una librería de análisis de sentimientos, donde permite realizar el calculo de subjectividad y polaridad de una frase completa sin tener que cortar la cadena en subcadenas, sirve para crear datasets de entrenamiento

In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob
import pandas as pd
import re

2. Llamamos a nuestro dataset para realizarle preprocesamiento

In [2]:
#Lo subimos directamente
from google.colab import files
uploaded = files.upload()
#o lo invocamos desde el directorio de datos

Saving covid_russia_vaccine_twits.csv to covid_russia_vaccine_twits (1).csv


3. Visualizamos que se subio correctamente y exploramos los campos del dataset en un dataframe

In [3]:
data = pd.read_csv('covid_russia_vaccine_twits.csv')
data.head(1)

,Date,Screen Name,Full Name,Tweet Text,Tweet ID,Link(s),Media,Location,Retweets,Favorites,App,Followers,Follows,Listed,Verfied,User Since,Location.1,Bio,Website,Timezone,Profile Image
0,12/08/2020 17:20,@geometryfan,Geometry Fan,Oops...and the 2 out of this 38 people are Put...,1293673804787593216,https://www.aerzteblatt.de/nachrichten/115504/...,NaN,NaN,0,0,Twitter for iPad,502,1981,30,NaN,5/08/2011,here,NaN,NaN,NaN,View


4. En un solo dataframe, nos centraremos en el texto del tweet
> Esto es crucial para el análisis de sentimientos, ya que lo que queremos obtener son distintos valores apartir de esta sola entrada, para realizar un buen preprocesamiento

In [4]:
tweets = pd.DataFrame(data['Tweet Text'])
tweets.head(10)

,Tweet Text
0,Oops...and the 2 out of this 38 people are Put...
1,Putin / tRump 2020 RT @smc429: @GOPChairwoman ...
2,HaHaHaHa Vladimir Putin You GO Boy @KremlinRus...
3,ohhhhhhhh putin RT @Samsinho_: Notre voiture a...
4,Not today Putin.
5,Putin’s “vaccine” must be the reason Donny has...
6,Most embarrassing use of The Oval Office EVER!...
7,For @realDonaldTrump : Russia. Putin. Bounties...
8,Vladimir Putin: Donald Trump vs Vladimir Putin...
9,Putin is reportedly looking to expand Russia's...


## Polaridad, Subjetividad, Clasificación de Tweets

5. Asignando polaridad, subjetividad y una clasificación al tweet
> TextBlob al estar basado en NLTK y sus 50 corpus (conjunto de textos), puede realizar un analisis de sentimiento rápido incluso por encima de los emojis o signos de puntación, analiza el tweet como una unidad textual.

In [5]:
#Esta función devuelve la subjectividad que determina del tweet completo
#Subjetividad: Se refiere a una medida objetiva que no involucra sentimiento
#Los posibles valores van de 0 a 1
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

#Esta función devuelve la polaridad del tweet completo
#Polaridad: Se refiere a la medida del sentimiento mismo
#Los posibles valores van de -1 a 1
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

#Clasificaremos según la polaridad, ya que es lo que nos interesa
#Si es menor a 0 es un tweet negativo
#Si es mayor a 0 es un tweet positivo
#Si es igual a 0 es un tweet neutro
def getClasification(text):
  if TextBlob(text).sentiment.polarity > 0:
    return 'Positive'
  elif TextBlob(text).sentiment.polarity == 0:
    return 'Neutral'
  else:
    return 'Negative'

6. Agregamos columnas a nuestro dataframe para poder ver los valores obtenidos de las funciones anteriormente declaradas, a su vez declaramos una columna "Label" que nos permitirá realizar la visualización de datos y la implementación de los modelos mas adelante.

In [6]:
tweets['Polarity'] = tweets['Tweet Text'].apply(getPolarity)
tweets['Subjectivity'] = tweets['Tweet Text'].apply(getSubjectivity)
tweets['Clasify'] = tweets['Tweet Text'].apply(getClasification)
tweets['Label'] = tweets['Clasify'].map({'Positive': 1, 'Negative': 0, 'Neutral': 2})
tweets

,Tweet Text,Polarity,Subjectivity,Clasify,Label
0,Oops...and the 2 out of this 38 people are Put...,0.000000,0.000000,Neutral,2
1,Putin / tRump 2020 RT @smc429: @GOPChairwoman ...,0.000000,0.000000,Neutral,2
2,HaHaHaHa Vladimir Putin You GO Boy @KremlinRus...,0.300000,0.450000,Positive,1
3,ohhhhhhhh putin RT @Samsinho_: Notre voiture a...,0.000000,0.000000,Neutral,2
4,Not today Putin.,0.000000,0.000000,Neutral,2
...,...,...,...,...,...
2891,Hopefully Drumpf will and maybe sit in on a 6t...,0.000000,0.000000,Neutral,2
2892,#LyingTed knew. He's just earning the money Pu...,0.045455,0.318182,Positive,1
2893,This whole thing reminds me of an episode of T...,0.150000,0.244444,Positive,1
2894,This is a Republican using the military to pan...,-0.060000,0.280000,Negative,0


## Limpieza de Tweets para Análisis de Sentimientos

7. Filtramos el dataframe anterior a solamente los tweets que sean positivos o negativos, ya que en estos Labels (0 y 1) se basará el proyecto de aqui en adelante

In [7]:
pos_neg_df = pd.DataFrame(tweets[tweets['Label'] != 2], columns=['Tweet Text', 'Label'])
pos_neg_df.head()

,Tweet Text,Label
2,HaHaHaHa Vladimir Putin You GO Boy @KremlinRus...,1
5,Putin’s “vaccine” must be the reason Donny has...,1
6,Most embarrassing use of The Oval Office EVER!...,1
8,Vladimir Putin: Donald Trump vs Vladimir Putin...,1
9,Putin is reportedly looking to expand Russia's...,1


8. A este nuevo dataframe le haremos limpieza al texto
> Se hará limpieza de caracteres especiales <b>usando str.replace</b> y pasandole como parametros la expresión regular y el valor a cambiar (puede ser en viseversa)<br>
> Se aplicará una función lambda para eliminar palabras cortas o monosílabas <b>incluyendo stopwords</b> <br>
> Se aplicará una función lambda para tokenizar el tweet (separar un string en substrings)
<br>
> <i>NOTA: Una función lambda es una función X que se define sin nombre y que realizará ciertas acciones, normalmente se usa para aplicar una función a un dataframe

In [8]:
# eliminando caracteres especiales, numeros, signos de puntuacion, etc
pos_neg_df['Clean Tweet'] = pos_neg_df['Tweet Text'].str.replace('[^a-zA-Z]+',' ')
# eliminando stopwords
pos_neg_df['Clean Tweet'] = pos_neg_df['Clean Tweet'].apply(lambda x: ' '.join([w.lower() for w in x.split() if len(w) > 2]))
# creando tokens o substrings
tokenized_message = pos_neg_df['Clean Tweet'].apply(lambda x: x.split())

In [9]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
# aplicamos Porter Stemmer a cada uno de los strings
tokenized_message = tokenized_message.apply(lambda x: [stemmer.stem(i) for i in x])

In [10]:
# juntamos los tokens en un solo string
for i in range(len(tokenized_message)):
    tokenized_message.iloc[i] = ' '.join(tokenized_message.iloc[i])

In [11]:
pos_neg_df['Clean Tweet']  = tokenized_message
pos_neg_df.head()

,Tweet Text,Label,Clean Tweet
2,HaHaHaHa Vladimir Putin You GO Boy @KremlinRus...,1,hahahaha vladimir putin you boy kremlinrussia ...
5,Putin’s “vaccine” must be the reason Donny has...,1,putin vaccin must the reason donni ha been sur...
6,Most embarrassing use of The Oval Office EVER!...,1,most embarrass use the oval offic ever well ex...
8,Vladimir Putin: Donald Trump vs Vladimir Putin...,1,vladimir putin donald trump vladimir putin vac...
9,Putin is reportedly looking to expand Russia's...,1,putin reportedli look expand russia presenc af...


## Ingeniería de características en análisis de sentimientos

9. Por cada string de un tweet, se le realizará ingeniería de características (Feature Engineering), que es un proceso de se genera caracteristicas adicionales a un tema, de tal manera de mejorar su estudio
> En análisis de sentimientos consideraremos las siguientes características adicionales:
> <br> <b>Tamaño de la cadena</b>
> <br> <b>Cantidad de palabras largas en la cadena</b>
> <br> <b>Cantidad de palabras cortas en la cadena</b>



In [12]:
pos_neg_df['Lenght'] = pos_neg_df['Tweet Text'].apply(lambda x : len(x) - x.count(" "))
pos_neg_df['Long Number'] = pos_neg_df['Tweet Text'].apply(lambda x : len(re.findall('\d{7,}',x)))
pos_neg_df['Short Number'] = pos_neg_df['Tweet Text'].apply(lambda x : len(re.findall('\d{4,6}',x)))

> Otra caracteristica también es la ponderación o porcentaje de sentimiento en el tweet dado por los signos de puntuación
> <br>
> <b>Punctuation</b> hace referencia a dicha ponderación

In [13]:
#De la libreria string, calculamos la ponderación de sentimiento agregado
#Esto se saca buscando en cada palabra del tweet, el count vendría a ser las veces de un simbolo encontrado en el texto
#Luego se lo promedia con la cantidad de palabras totales, excepto los whitespaces.
import string
def count_punct (text):
    count = sum([1 for x in text if x in string.punctuation])
    pp = round(100*count/(len(text)-text.count(" ")),3)
    return pp

In [14]:
#Aplicamos la función anterior al Tweet original (no al Tweet limpio)
pos_neg_df['Punctuation'] = pos_neg_df['Tweet Text'].apply(lambda x : count_punct(x))

> Otra caracteristica también es si algún tweet invoca a alguna pagina web, de tal manera que dicha web tenga alguna relación con el tema estudiado por el caso
> <br>
> <b>Website</b> hace referencia a dicho descubrimiento

In [15]:
#Creamos la función que analizará el tweet original (no el tweet limpio) y buscará algun indicio de que el mensaje incluye algun link
#Si es asi, con un booleano determinamos si es verdadero o falso, este dato lo usaremos para la parte de visualización y modelos
def website(text):
    if (len(re.findall('www|http|com|\.co',text))>0):
        return 1
    else:
        return 0

In [16]:
#Aplicamos la funcion web anterior a nuestro dataframe
pos_neg_df['WebSite'] = pos_neg_df['Tweet Text'].apply(lambda x : website(x))
pos_neg_df.head(10)

,Tweet Text,Label,Clean Tweet,Lenght,Long Number,Short Number,Punctuation,WebSite
2,HaHaHaHa Vladimir Putin You GO Boy @KremlinRus...,1,hahahaha vladimir putin you boy kremlinrussia ...,292,0,0,5.137,0
5,Putin’s “vaccine” must be the reason Donny has...,1,putin vaccin must the reason donni ha been sur...,186,0,2,9.677,0
6,Most embarrassing use of The Oval Office EVER!...,1,most embarrass use the oval offic ever well ex...,121,0,0,3.306,0
8,Vladimir Putin: Donald Trump vs Vladimir Putin...,1,vladimir putin donald trump vladimir putin vac...,80,0,0,3.750,0
9,Putin is reportedly looking to expand Russia's...,1,putin reportedli look expand russia presenc af...,137,0,0,4.380,0
10,CNN and MSNBC keep it up ! Traitor trump and P...,1,cnn and msnbc keep traitor trump and putin gop...,184,0,0,3.261,0
12,Kamala should bring Pence a gift. Like Putin g...,1,kamala should bring penc gift like putin gave ...,325,0,3,4.000,0
13,#Putin is well aware of the essential #implica...,1,putin well awar the essenti implic success cov...,324,0,0,9.259,0
14,"More Deficit, @realDonaldTrump is just Bankrup...",1,more deficit realdonaldtrump just bankrupt the...,258,0,0,3.876,1
15,"Of all state employees, the police, law enforc...",0,all state employe the polic law enforc agenc a...,180,0,0,4.444,0


In [17]:
#Guardamos los resultados en un nuevo dataset para el uso en la parte de visualizacion y modelos
pos_neg_df.to_csv('vacuna_dataset_preprocesado.csv', index=False)